In [ ]:
from mqt.ddsim import DDSIMProvider

provider = DDSIMProvider()

from qiskit import QuantumCircuit

circ = QuantumCircuit(2)
circ.h(0)
circ.cx(0, 1)
circ.measure_all()
circ.draw(output="mpl", style="iqp")

## Usage as Qiskit Backends

The DDSIMProvider currently has seven backends

- `QasmSimulator` simulates a circuit and generates the given number of shots
- `StatevectorSimulator` simulates the circuit and returns the statevector
- `HybridQasmSimulator` simulates a circuit in parallel using a hybrid Schrödinger-Feynman technique and generates the given number of shots
- `HybridStatevectorSimulator` simulates the circuit in parallel using a hybrid Schrödinger-Feynman technique and returns the statevector
- `PathQasmSimulator` simulates a circuit by potential using a different order of multiplying operations and operation/state and generates the requested number of shots
- `PathStatevectorSimulator` simulates a circuit by potential using a different order of multiplying operations and operation/state and returns the statevector
- `UnitarySimulator` constructs the unitary functionality of a circuit and returns the corresponding unitary matrix

### PathQasmSimulator for Sampling

In [ ]:
backend = provider.get_backend("path_sim_qasm_simulator")
print(f"Backend version: {backend.backend_version}")
job = backend.run(circ, shots=100000)  # uses the sequential strategy b
result = job.result()
counts = result.get_counts(circ)
print(counts)